In [1]:
import pickle
import pdfminer
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine
import re
import urllib
import glob
import os
from random import randint
from time import sleep
import pandas as pd

import chromedriver_binary

In [2]:
#tuple is (page,bbox[0],bbox[1],bbox[2],bbox[3])
lookup_table = {
    (0,18,672,212,746) : 'api_num',
    (0,18,417,209,561) : 'file_purpose', 
    (0,18,132,383,390) : 'well_info',
    (0,308,482,505,529) : 'comp_date',
    (1,18,762,530,872) : 'test_info',
    (1,18,681,130,726) : 'oil_water_24',
    (1,244,697,415,743) : 'gas_24',
    (1,309,265,347,276) : 'depth_from', #309.040,260.430,347.960,271.990
    (1,531,265,567,276) : 'depth_to',   #531.430,260.430,567.570,271.990
    (0,248,208,563,404) : 'md', #248.000,208.243,563.359,404.333
    (2,389,833,413,846) : 'pressure'         #389.000,833.273,413.464,845.989 
}

In [3]:
def textbox_lookup(file,lookup):
    parser = PDFParser(file)
    doc = PDFDocument()
    parser.set_document(doc)
    doc.set_parser(parser)
    doc.initialize('')
    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    laparams.char_margin = 1.0
    laparams.word_margin = 1.0
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    scrape_dict = {}
    score_table = {}
    page_count = -1
    for page in doc.get_pages():
        page_count += 1
        interpreter.process_page(page)
        layout = device.get_result()
        for lt_obj in layout:
            if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
                obj_address = (page_count,
                               int(lt_obj.bbox[0]),
                               int(lt_obj.bbox[1]),
                               int(lt_obj.bbox[2]),
                               int(lt_obj.bbox[3]))
                
                for item in lookup.keys():
                    close_score = ((item[0] - obj_address[0])**2*1000000000 +
                                   (item[1] - obj_address[1])**2 + 
                                   (item[2] - obj_address[2])**2 +
                                   (item[3] - obj_address[3])**2 +
                                   (item[4] - obj_address[4])**2)
                    if item in score_table.keys():
                        if close_score < score_table[item]:
                            score_table[item] = close_score
                            scrape_dict[lookup[item]] = lt_obj.get_text()
                    else:
                        score_table[item] = close_score
                        scrape_dict[lookup[item]] = lt_obj.get_text()
                

    return scrape_dict

In [4]:
def clean_strings(string_dict):
    comp_keys = ['api_num','purpose', 'comp_type', 'well_type',
                 'comp_date','tvd','test_date',
                 'test_method','choke','ip24_oil',
                 'ip24_gas','ip24_water','ll','md','pressure']
    
    comp_dict = {key:None for key in comp_keys}
    try:
        comp_dict['api_num'] = re.search(r'API No\.:\n(.*?)\n',string_dict['api_num']).group(1)
    except:
        comp_dict['api_num'] = None
    try:
        comp_dict['purpose'] = re.search(r'Purpose of filing:\n(.*?)\n',string_dict['file_purpose']).group(1)
    except:
        comp_dict['purpose'] = None
    try:
        comp_dict['comp_type'] = re.search(r'Type of completion:\n(.*?)\n',string_dict['file_purpose']).group(1)
    except:
        comp_dict['comp_type'] = None
    try:
        comp_dict['well_type'] = re.search(r'Well Type:\n(.*?)\n',string_dict['file_purpose']).group(1)
    except:
        comp_dict['well_type'] = None
    try:
        comp_dict['comp_date'] = re.search(r'Completion or Recompletion Date:\nDate\n(.*?)\n',string_dict['comp_date']).group(1)
    except:
        comp_dict['comp_date'] = None
    try:
        comp_dict['tvd'] = re.search(r'Total depth TVD \(ft\.\):\n(.*?)\n',string_dict['well_info']).group(1)
    except:
        comp_dict['tvd'] = None
    try:
        comp_dict['test_date'] = re.search(r'Date of test:\n(.*?)\n',string_dict['test_info']).group(1)
    except:
        comp_dict['test_date'] = None
    try:
        comp_dict['test_method'] = re.search(r'Production method:\n(.*?)\n',string_dict['test_info']).group(1)
    except:
        comp_dict['test_method'] = None
    try:
        comp_dict['choke'] = re.search(r'Choke size:\n(.*?)\n',string_dict['test_info']).group(1)
    except:
        comp_dict['choke'] = None
    try:
        comp_dict['ip24_oil'] = re.search(r'Oil \(BBLS\):\n(.*?)\n',string_dict['oil_water_24']).group(1)
    except:
        comp_dict['ip24_oil'] = None
    try:
        comp_dict['ip24_water'] = re.search(r'Water \(BBLS\):\n(.*?)\n',string_dict['oil_water_24']).group(1)
    except:
        comp_dict['ip24_water'] = None
    try:
        comp_dict['pressure'] = float(re.findall(r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", string_dict['pressure'])[-1])
    except:
        comp_dict['pressure'] = None
    try:
        comp_dict['ip24_gas'] = re.search(r'Gas \(MCF\):\n(.*?)\n',string_dict['gas_24']).group(1)
    except:
        comp_dict['ip24_gas'] = None
    try:
        depth_from = float(re.findall(r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", string_dict['depth_from'])[-1])
        depth_to = float(re.findall(r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", string_dict['depth_to'])[-1])

        ll = depth_to - depth_from

        if ll > 0:
            comp_dict['ll'] = ll
    except:
        comp_dict['ll'] = None
        
    try:
        comp_dict['md'] = re.search(r'Total depth MD \(ft\.\):\n(.*?)\n',string_dict['md']).group(1)
    except:
        comp_dict['md'] = None
    


    
    return comp_dict

In [5]:
def scrape_pdfs(dir_path):
    file_list = glob.glob(dir_path+'/*')
    report_list = []
    for fp in file_list:
        with open(fp,'rb') as pdf:
            print(fp)
            report_list.append(clean_strings(textbox_lookup(pdf,lookup_table)))
    return pd.DataFrame(report_list)

In [6]:
def to_pickle(var,name,dir_name='pickle'):
    with open('{}/{}.pickle'.format(dir_name,name), 'wb') as to_write:
        pickle.dump(var, to_write)
    

In [11]:
n_midland_pdfs = scrape_pdfs('data/pdfs/n_midland')
n_midland_pdfs.head()

data/pdfs/n_midland/207532.pdf
data/pdfs/n_midland/208629.pdf


data/pdfs/n_midland/199188.pdf


data/pdfs/n_midland/150377.pdf
data/pdfs/n_midland/189402.pdf
data/pdfs/n_midland/109564.pdf
data/pdfs/n_midland/145752.pdf
data/pdfs/n_midland/107788.pdf
data/pdfs/n_midland/202648.pdf


data/pdfs/n_midland/131574.pdf
data/pdfs/n_midland/118680.pdf
data/pdfs/n_midland/160049.pdf
data/pdfs/n_midland/122291.pdf
data/pdfs/n_midland/142565.pdf
data/pdfs/n_midland/158529.pdf


data/pdfs/n_midland/188250.pdf
data/pdfs/n_midland/114858.pdf
data/pdfs/n_midland/207644.pdf
data/pdfs/n_midland/199229.pdf


...

data/pdfs/n_midland/113055.pdf
data/pdfs/n_midland/212495.pdf


data/pdfs/n_midland/130667.pdf
data/pdfs/n_midland/160050.pdf


data/pdfs/n_midland/198933.pdf
data/pdfs/n_midland/197619.pdf
data/pdfs/n_midland/105421.pdf
data/pdfs/n_midland/207517.pdf
data/pdfs/n_midland/203003.pdf


data/pdfs/n_midland/115829.pdf


,api_num,choke,comp_date,comp_type,ip24_gas,ip24_oil,ip24_water,ll,md,pressure,purpose,test_date,test_method,tvd,well_type
0,42-227-39481,Oil produced prior to test:,02/02/2018,New Well,Casing Pressure:,Oil Gravity - API - 60.:,None,10463.0,18552,9207.0,Well Record Only,None,Choke size:,7580,Producing
1,42-227-39577,29/64,04/19/2018,New Well,589,1073.0,311,7172.0,15885,9390.0,Initial Potential,02/02/2019,Pumping,8426,Producing
2,42-227-39182,Was swab used during this test?,07/14/2017,New Well,559,665.3,875,9251.0,17813,9212.0,Initial Potential,06/11/2018,Pumping,7909,Producing
3,42-227-38385,Was swab used during this test?,11/19/2014,New Well,613,808.0,911,4906.0,13201,3273.0,Initial Potential,09/23/2015,Pumping,7586,Producing
4,42-227-38885,Was swab used during this test?,10/31/2016,Other/Recompletion,783,932.0,4223,7828.0,15511,9157.0,Initial Potential,10/24/2017,Pumping,7288,Producing


In [13]:
to_pickle(n_midland_pdfs,'n_midland_pdfs')

In [10]:
to_pickle(scrape_pdfs('data/pdfs/ef_all'),'ef_pdfs')

data/pdfs/ef_all/170716.pdf
data/pdfs/ef_all/170070.pdf
data/pdfs/ef_all/56941.pdf
data/pdfs/ef_all/208615.pdf
data/pdfs/ef_all/181937.pdf
data/pdfs/ef_all/215650.pdf
data/pdfs/ef_all/210728.pdf
data/pdfs/ef_all/172883.pdf
data/pdfs/ef_all/217709.pdf
data/pdfs/ef_all/190806.pdf
data/pdfs/ef_all/177723.pdf
data/pdfs/ef_all/190812.pdf
data/pdfs/ef_all/103908.pdf
data/pdfs/ef_all/164978.pdf
data/pdfs/ef_all/186043.pdf
data/pdfs/ef_all/150363.pdf
data/pdfs/ef_all/203783.pdf
data/pdfs/ef_all/184898.pdf
data/pdfs/ef_all/195003.pdf
data/pdfs/ef_all/208826.pdf
data/pdfs/ef_all/196278.pdf
data/pdfs/ef_all/162110.pdf
data/pdfs/ef_all/170500.pdf
data/pdfs/ef_all/162662.pdf
data/pdfs/ef_all/206360.pdf
data/pdfs/ef_all/75418.pdf
data/pdfs/ef_all/205669.pdf
data/pdfs/ef_all/157197.pdf
data/pdfs/ef_all/177247.pdf
data/pdfs/ef_all/157183.pdf
data/pdfs/ef_all/173753.pdf
data/pdfs/ef_all/157168.pdf
data/pdfs/ef_all/180624.pdf
data/pdfs/ef_all/208205.pdf
data/pdfs/ef_all/194080.pdf
data/pdfs/ef_all/17802

data/pdfs/ef_all/51975.pdf
data/pdfs/ef_all/202305.pdf
data/pdfs/ef_all/177722.pdf
data/pdfs/ef_all/190807.pdf
data/pdfs/ef_all/217708.pdf
data/pdfs/ef_all/151901.pdf
data/pdfs/ef_all/208172.pdf
data/pdfs/ef_all/208614.pdf
data/pdfs/ef_all/212894.pdf
data/pdfs/ef_all/205330.pdf
data/pdfs/ef_all/215135.pdf
data/pdfs/ef_all/205326.pdf
data/pdfs/ef_all/158927.pdf
data/pdfs/ef_all/192948.pdf
data/pdfs/ef_all/188938.pdf
data/pdfs/ef_all/156922.pdf
data/pdfs/ef_all/190811.pdf
data/pdfs/ef_all/190805.pdf
data/pdfs/ef_all/177720.pdf
data/pdfs/ef_all/178375.pdf
data/pdfs/ef_all/175889.pdf
data/pdfs/ef_all/210930.pdf
data/pdfs/ef_all/53635.pdf
data/pdfs/ef_all/168833.pdf
data/pdfs/ef_all/192021.pdf
data/pdfs/ef_all/188723.pdf
data/pdfs/ef_all/192009.pdf
data/pdfs/ef_all/209291.pdf
data/pdfs/ef_all/194849.pdf
data/pdfs/ef_all/201356.pdf
data/pdfs/ef_all/180800.pdf
data/pdfs/ef_all/173750.pdf
data/pdfs/ef_all/202677.pdf
data/pdfs/ef_all/174981.pdf
data/pdfs/ef_all/146099.pdf
data/pdfs/ef_all/17374

data/pdfs/ef_all/155635.pdf
data/pdfs/ef_all/151335.pdf
data/pdfs/ef_all/194055.pdf
data/pdfs/ef_all/196642.pdf
data/pdfs/ef_all/187997.pdf
data/pdfs/ef_all/207849.pdf
data/pdfs/ef_all/152600.pdf
data/pdfs/ef_all/192356.pdf
data/pdfs/ef_all/167877.pdf
data/pdfs/ef_all/215281.pdf
data/pdfs/ef_all/156472.pdf
data/pdfs/ef_all/152614.pdf
data/pdfs/ef_all/167122.pdf
data/pdfs/ef_all/187968.pdf
data/pdfs/ef_all/162728.pdf
data/pdfs/ef_all/209131.pdf
data/pdfs/ef_all/202064.pdf
data/pdfs/ef_all/193114.pdf
data/pdfs/ef_all/82045.pdf
data/pdfs/ef_all/188246.pdf
data/pdfs/ef_all/85702.pdf
data/pdfs/ef_all/166348.pdf
data/pdfs/ef_all/171146.pdf
data/pdfs/ef_all/177537.pdf
data/pdfs/ef_all/174980.pdf
data/pdfs/ef_all/171608.pdf
data/pdfs/ef_all/202138.pdf
data/pdfs/ef_all/162674.pdf
data/pdfs/ef_all/175308.pdf
data/pdfs/ef_all/169053.pdf
data/pdfs/ef_all/182569.pdf
data/pdfs/ef_all/192034.pdf
data/pdfs/ef_all/187439.pdf
data/pdfs/ef_all/177721.pdf
data/pdfs/ef_all/181909.pdf
data/pdfs/ef_all/21210

data/pdfs/ef_all/178665.pdf
data/pdfs/ef_all/210213.pdf
data/pdfs/ef_all/159654.pdf
data/pdfs/ef_all/204299.pdf
data/pdfs/ef_all/198008.pdf
data/pdfs/ef_all/154416.pdf
data/pdfs/ef_all/200799.pdf
data/pdfs/ef_all/192525.pdf
data/pdfs/ef_all/191004.pdf
data/pdfs/ef_all/211864.pdf
data/pdfs/ef_all/176847.pdf
data/pdfs/ef_all/193161.pdf
data/pdfs/ef_all/165768.pdf
data/pdfs/ef_all/215237.pdf
data/pdfs/ef_all/175209.pdf
data/pdfs/ef_all/208272.pdf
data/pdfs/ef_all/165754.pdf
data/pdfs/ef_all/194961.pdf
data/pdfs/ef_all/179369.pdf
data/pdfs/ef_all/194034.pdf
data/pdfs/ef_all/103348.pdf
data/pdfs/ef_all/199276.pdf
data/pdfs/ef_all/11704.pdf
data/pdfs/ef_all/201295.pdf
data/pdfs/ef_all/184233.pdf
data/pdfs/ef_all/209187.pdf
data/pdfs/ef_all/198154.pdf
data/pdfs/ef_all/57590.pdf
data/pdfs/ef_all/204138.pdf
data/pdfs/ef_all/199909.pdf
data/pdfs/ef_all/190093.pdf
data/pdfs/ef_all/197930.pdf
data/pdfs/ef_all/169422.pdf
data/pdfs/ef_all/214303.pdf
data/pdfs/ef_all/158628.pdf
data/pdfs/ef_all/15617

data/pdfs/ef_all/214664.pdf
data/pdfs/ef_all/167632.pdf
data/pdfs/ef_all/177353.pdf
data/pdfs/ef_all/168298.pdf
data/pdfs/ef_all/199261.pdf
data/pdfs/ef_all/201296.pdf
data/pdfs/ef_all/201282.pdf
data/pdfs/ef_all/199275.pdf
data/pdfs/ef_all/180042.pdf
data/pdfs/ef_all/200822.pdf
data/pdfs/ef_all/205972.pdf
data/pdfs/ef_all/155131.pdf
data/pdfs/ef_all/199249.pdf
data/pdfs/ef_all/175008.pdf
data/pdfs/ef_all/162574.pdf
data/pdfs/ef_all/164677.pdf
data/pdfs/ef_all/58878.pdf
data/pdfs/ef_all/207432.pdf
data/pdfs/ef_all/184783.pdf
data/pdfs/ef_all/174458.pdf
data/pdfs/ef_all/217190.pdf
data/pdfs/ef_all/190047.pdf
data/pdfs/ef_all/206089.pdf
data/pdfs/ef_all/184998.pdf
data/pdfs/ef_all/157518.pdf
data/pdfs/ef_all/206937.pdf
data/pdfs/ef_all/203867.pdf
data/pdfs/ef_all/155327.pdf
data/pdfs/ef_all/208932.pdf
data/pdfs/ef_all/21388.pdf


data/pdfs/ef_all/156148.pdf
data/pdfs/ef_all/155333.pdf
data/pdfs/ef_all/59541.pdf
data/pdfs/ef_all/167380.pdf
data/pdfs/ef_all/208927.pdf
data/pdfs/ef_all/203866.pdf
data/pdfs/ef_all/206088.pdf
data/pdfs/ef_all/206936.pdf
data/pdfs/ef_all/157519.pdf
data/pdfs/ef_all/192651.pdf
data/pdfs/ef_all/215786.pdf
data/pdfs/ef_all/167343.pdf
data/pdfs/ef_all/202211.pdf
data/pdfs/ef_all/210173.pdf
data/pdfs/ef_all/174465.pdf
data/pdfs/ef_all/166061.pdf
data/pdfs/ef_all/184782.pdf
data/pdfs/ef_all/162575.pdf
data/pdfs/ef_all/175009.pdf
data/pdfs/ef_all/179633.pdf
data/pdfs/ef_all/158825.pdf
data/pdfs/ef_all/209185.pdf
data/pdfs/ef_all/205973.pdf
data/pdfs/ef_all/166288.pdf
data/pdfs/ef_all/157733.pdf
data/pdfs/ef_all/194036.pdf
data/pdfs/ef_all/167627.pdf
data/pdfs/ef_all/194977.pdf
data/pdfs/ef_all/214671.pdf
data/pdfs/ef_all/206261.pdf
data/pdfs/ef_all/175237.pdf
data/pdfs/ef_all/167633.pdf
data/pdfs/ef_all/214895.pdf
data/pdfs/ef_all/198022.pdf
data/pdfs/ef_all/176845.pdf
data/pdfs/ef_all/1843

data/pdfs/ef_all/216302.pdf
data/pdfs/ef_all/175635.pdf
data/pdfs/ef_all/162436.pdf
data/pdfs/ef_all/174532.pdf
data/pdfs/ef_all/203064.pdf
data/pdfs/ef_all/209575.pdf
data/pdfs/ef_all/210756.pdf
data/pdfs/ef_all/173275.pdf
data/pdfs/ef_all/192921.pdf
data/pdfs/ef_all/181785.pdf
data/pdfs/ef_all/185508.pdf
data/pdfs/ef_all/196574.pdf
data/pdfs/ef_all/184602.pdf
data/pdfs/ef_all/188038.pdf
data/pdfs/ef_all/194439.pdf
data/pdfs/ef_all/166877.pdf
data/pdfs/ef_all/193430.pdf
data/pdfs/ef_all/210795.pdf
data/pdfs/ef_all/145704.pdf
data/pdfs/ef_all/170556.pdf
data/pdfs/ef_all/177239.pdf
data/pdfs/ef_all/180869.pdf
data/pdfs/ef_all/179572.pdf
data/pdfs/ef_all/210568.pdf
data/pdfs/ef_all/160989.pdf
data/pdfs/ef_all/152907.pdf
data/pdfs/ef_all/212425.pdf
data/pdfs/ef_all/200747.pdf
data/pdfs/ef_all/209005.pdf
data/pdfs/ef_all/201459.pdf
data/pdfs/ef_all/216870.pdf
data/pdfs/ef_all/174095.pdf
data/pdfs/ef_all/184366.pdf
data/pdfs/ef_all/172396.pdf
data/pdfs/ef_all/168480.pdf
data/pdfs/ef_all/178

data/pdfs/ef_all/217189.pdf
data/pdfs/ef_all/155302.pdf
data/pdfs/ef_all/213304.pdf
data/pdfs/ef_all/202209.pdf
data/pdfs/ef_all/197902.pdf
data/pdfs/ef_all/174455.pdf
data/pdfs/ef_all/169404.pdf
data/pdfs/ef_all/185484.pdf
data/pdfs/ef_all/175005.pdf
data/pdfs/ef_all/178253.pdf
data/pdfs/ef_all/175011.pdf
data/pdfs/ef_all/209412.pdf
data/pdfs/ef_all/202547.pdf
data/pdfs/ef_all/167415.pdf
data/pdfs/ef_all/192463.pdf
data/pdfs/ef_all/151428.pdf
data/pdfs/ef_all/104367.pdf
data/pdfs/ef_all/173851.pdf
data/pdfs/ef_all/178910.pdf
data/pdfs/ef_all/209189.pdf
data/pdfs/ef_all/196639.pdf
data/pdfs/ef_all/193999.pdf
data/pdfs/ef_all/195456.pdf
data/pdfs/ef_all/199278.pdf
data/pdfs/ef_all/173879.pdf
data/pdfs/ef_all/185651.pdf
data/pdfs/ef_all/160150.pdf
data/pdfs/ef_all/207615.pdf
data/pdfs/ef_all/199287.pdf
data/pdfs/ef_all/152848.pdf
data/pdfs/ef_all/170419.pdf
data/pdfs/ef_all/165772.pdf
data/pdfs/ef_all/56496.pdf
data/pdfs/ef_all/175207.pdf
data/pdfs/ef_all/171075.pdf
data/pdfs/ef_all/1903

data/pdfs/ef_all/201823.pdf
data/pdfs/ef_all/181043.pdf
data/pdfs/ef_all/205112.pdf
data/pdfs/ef_all/214779.pdf
data/pdfs/ef_all/169070.pdf
data/pdfs/ef_all/165104.pdf
data/pdfs/ef_all/185796.pdf
data/pdfs/ef_all/169058.pdf
data/pdfs/ef_all/170509.pdf
data/pdfs/ef_all/216146.pdf
data/pdfs/ef_all/171603.pdf
data/pdfs/ef_all/174948.pdf
data/pdfs/ef_all/154440.pdf
data/pdfs/ef_all/90338.pdf
data/pdfs/ef_all/195208.pdf
data/pdfs/ef_all/184463.pdf
data/pdfs/ef_all/210286.pdf
data/pdfs/ef_all/188517.pdf
data/pdfs/ef_all/206396.pdf
data/pdfs/ef_all/187977.pdf
data/pdfs/ef_all/166223.pdf
data/pdfs/ef_all/175263.pdf
data/pdfs/ef_all/185184.pdf
data/pdfs/ef_all/206221.pdf
data/pdfs/ef_all/163415.pdf
data/pdfs/ef_all/216967.pdf
data/pdfs/ef_all/178960.pdf
data/pdfs/ef_all/184271.pdf
data/pdfs/ef_all/199220.pdf
data/pdfs/ef_all/180771.pdf
data/pdfs/ef_all/207856.pdf
data/pdfs/ef_all/215063.pdf
data/pdfs/ef_all/201758.pdf
data/pdfs/ef_all/203629.pdf
data/pdfs/ef_all/191481.pdf
data/pdfs/ef_all/1782

data/pdfs/ef_all/185197.pdf
data/pdfs/ef_all/187958.pdf
data/pdfs/ef_all/206226.pdf
data/pdfs/ef_all/205049.pdf
data/pdfs/ef_all/194924.pdf
data/pdfs/ef_all/184470.pdf
data/pdfs/ef_all/154447.pdf
data/pdfs/ef_all/155981.pdf
data/pdfs/ef_all/157172.pdf
data/pdfs/ef_all/188262.pdf
data/pdfs/ef_all/200086.pdf
data/pdfs/ef_all/208357.pdf
data/pdfs/ef_all/169711.pdf
data/pdfs/ef_all/200079.pdf
data/pdfs/ef_all/201367.pdf
data/pdfs/ef_all/30162.pdf
data/pdfs/ef_all/201429.pdf
data/pdfs/ef_all/202134.pdf
data/pdfs/ef_all/180825.pdf
data/pdfs/ef_all/182565.pdf
data/pdfs/ef_all/208828.pdf
data/pdfs/ef_all/203799.pdf
data/pdfs/ef_all/187347.pdf
data/pdfs/ef_all/181044.pdf
data/pdfs/ef_all/217713.pdf
data/pdfs/ef_all/201165.pdf
data/pdfs/ef_all/190808.pdf
data/pdfs/ef_all/178350.pdf
data/pdfs/ef_all/164779.pdf
data/pdfs/ef_all/47856.pdf
data/pdfs/ef_all/206622.pdf
data/pdfs/ef_all/212125.pdf
data/pdfs/ef_all/192945.pdf
data/pdfs/ef_all/193481.pdf
data/pdfs/ef_all/209261.pdf
data/pdfs/ef_all/21771

In [12]:
to_pickle(scrape_pdfs('data/pdfs/midland_all'),'midland_all_pdfs')

data/pdfs/midland_all/188913.pdf
data/pdfs/midland_all/204749.pdf
data/pdfs/midland_all/205331.pdf


data/pdfs/midland_all/184697.pdf
data/pdfs/midland_all/194484.pdf
data/pdfs/midland_all/211344.pdf


data/pdfs/midland_all/207532.pdf
data/pdfs/midland_all/164011.pdf
data/pdfs/midland_all/208601.pdf
data/pdfs/midland_all/162460.pdf


data/pdfs/midland_all/205325.pdf


data/pdfs/midland_all/166606.pdf
data/pdfs/midland_all/213547.pdf


...

data/pdfs/midland_all/201614.pdf
data/pdfs/midland_all/205472.pdf


data/pdfs/midland_all/169262.pdf
data/pdfs/midland_all/203003.pdf


data/pdfs/midland_all/209260.pdf
data/pdfs/midland_all/204744.pdf
data/pdfs/midland_all/206635.pdf


data/pdfs/midland_all/197180.pdf
data/pdfs/midland_all/206621.pdf
data/pdfs/midland_all/153868.pdf
data/pdfs/midland_all/168626.pdf
data/pdfs/midland_all/205328.pdf


data/pdfs/midland_all/178353.pdf


In [105]:
def box_print(file):
    parser = PDFParser(file)
    doc = PDFDocument()
    parser.set_document(doc)
    doc.set_parser(parser)
    doc.initialize('')
    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    laparams.char_margin = 1.0
    laparams.word_margin = 1.0
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    scrape_dict = {}
    page_count = -1
    for page in doc.get_pages():
        page_count += 1
        interpreter.process_page(page)
        layout = device.get_result()
        for lt_obj in layout:
            if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
                print(lt_obj)
    return

file = open('/Users/alexanderparker/coding/frack-fluid-analysis/data/pdfs/42329417450000-2.pdf','rb')


box_print(file)

<LTTextBoxHorizontal(0) 201.270,914.762,444.730,931.422 'RAILROAD COMMISSION OF TEXAS\n'>
<LTTextBoxHorizontal(1) 250.800,867.156,396.204,909.366 '1701 N. Congress\nP.O. Box 12967\nAustin, Texas 78701-2967\n'>
<LTTextBoxHorizontal(2) 467.000,864.243,537.290,909.333 'Status:\nDate:\nTracking No.:\n'>
<LTTextBoxHorizontal(3) 570.450,914.762,635.004,931.422 'Form W-2\n'>
<LTTextBoxHorizontal(4) 596.480,897.587,635.000,907.991 'Approved\n'>
<LTTextBoxHorizontal(5) 589.960,865.587,635.004,890.991 '12/18/2018\n202919\n'>
<LTTextBoxHorizontal(6) 39.970,842.762,607.026,859.422 'OIL WELL POTENTIAL TEST, COMPLETION OR RECOMPLETION REPORT, AND LOG\n'>
<LTTextBoxHorizontal(7) 18.000,775.243,115.185,803.333 'Operator Name:\nOperator Address:\n'>
<LTTextBoxHorizontal(8) 107.000,776.587,334.591,801.991 'CHEVRON U. S. A. INC.\n6301 DEAUVILLE BLVD  MIDLAND, TX 79706-0000\n'>
<LTTextBoxHorizontal(9) 253.500,790.243,396.500,818.333 'OPERATOR INFORMATION\nOperator No.:\n'>
<LTTextBoxHorizontal(10) 401.000

<LTTextBoxHorizontal(0) 18.000,924.243,545.624,952.333 'ACID, FRACTURE, CEMENT SQUEEZE, CAST IRON BRIDGE PLUG, RETAINER, ETC.\nWas hydraulic fracturing treatment performed?\nYes\n'>
<LTTextBoxHorizontal(1) 26.390,739.273,49.611,768.133 'Row\n1\n'>
<LTTextBoxHorizontal(2) 61.440,739.273,155.567,768.133 'Type of Operation\nCement Squeeze\n'>
<LTTextBoxHorizontal(3) 222.300,739.273,449.703,768.133 'Amount and Kind of Material Used\n189 SACKS; 235 CU. FT., CLASS H CEMENT\n'>
<LTTextBoxHorizontal(4) 501.940,739.273,596.056,768.133 'Depth Interval (ft.)\n8324\n9047\n'>
<LTTextBoxHorizontal(5) 18.000,877.243,243.555,909.333 'Is well equipped with a downhole actuation\nsleeve?\nNo\n'>
<LTTextBoxHorizontal(6) 18.000,834.243,264.334,866.333 'Production casing test pressure (PSIG) prior to\nhydraulic fracturing treatment:\n'>
<LTTextBoxHorizontal(7) 18.000,795.443,291.229,821.333 'Has the hydraulic fracturing fluid disclosure been\nreported to FracFocus disclosure registry (SWR29)?\n'>
<LTTextBox